In [ ]:
import sys
sys.path.append('../src/ilab_group_12_1_fall_detection') 

In [1]:
import pandas as pd

In [3]:
df = pd.read_pickle("../../data/interim/df_freq_features.pkl")

In [7]:
df

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,...,acc_z_entropy,gyro_x_fft,gyro_x_psd,gyro_x_entropy,gyro_y_fft,gyro_y_psd,gyro_y_entropy,gyro_z_fft,gyro_z_psd,gyro_z_entropy
2,-0.068994,-6.987095,5.288118,1.340131,0.254273,0.553948,BSC,1.0,1.0,32.0,...,0.552713,2.237175,0.000722,0.920616,0.363307,0.000888,1.282563,0.614725,0.000267,1.354238
3,-0.470804,-1.801783,8.929937,1.701367,0.193919,0.328752,BSC,1.0,1.0,32.0,...,0.697146,1.886452,0.000581,0.905243,0.727568,0.000668,1.175494,1.138423,0.000603,1.415926
4,-9.901018,13.441229,7.822155,-0.351828,0.343581,-0.851439,BSC,1.0,1.0,32.0,...,1.074713,3.329770,0.005098,1.203505,1.706857,0.000565,0.936669,3.093796,0.002285,0.577150
5,-5.150726,6.613287,3.195224,-0.107008,-1.118601,-0.638797,BSC,1.0,1.0,32.0,...,1.461785,9.083104,0.043227,1.271543,2.602335,0.005232,1.358140,3.267480,0.006812,0.745639
6,-4.931030,5.574285,6.261700,-0.019441,0.012431,0.115133,BSC,1.0,1.0,32.0,...,1.176317,1.104385,0.000710,1.366146,3.538939,0.009700,1.667364,0.830726,0.003255,1.440714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169385,0.586223,5.443400,1.029336,-0.737727,1.443459,1.335214,WAL,67.0,1.0,23.0,...,1.343248,7.967362,0.019501,0.917517,3.978832,0.003662,1.133685,9.439387,0.030438,0.913799
169386,2.286937,11.130139,-1.195925,1.260658,-0.457721,-0.981431,WAL,67.0,1.0,23.0,...,1.502760,7.253110,0.017147,0.899968,6.142548,0.022069,1.258938,8.046604,0.023517,0.994088
169387,-5.076769,14.152212,2.039380,-2.541764,-1.378478,2.412521,WAL,67.0,1.0,23.0,...,1.377740,5.479225,0.007237,1.008082,3.459003,0.006143,1.726653,5.018624,0.007999,1.087638
169388,1.671382,11.554947,-1.548809,0.596082,-1.174618,-0.679053,WAL,67.0,1.0,23.0,...,1.449325,7.200013,0.037785,1.189187,3.266761,0.013842,1.523287,7.761068,0.044975,1.151021


## Data Splitting

In [8]:
df_train = df.drop(["subject_id", "trial"], axis = 1)

In [10]:
X = df_train.drop("label", axis =1)
y = df_train["label"]

In [22]:
X

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,age,height,weight,gender,...,acc_z_entropy,gyro_x_fft,gyro_x_psd,gyro_x_entropy,gyro_y_fft,gyro_y_psd,gyro_y_entropy,gyro_z_fft,gyro_z_psd,gyro_z_entropy
2,-0.068994,-6.987095,5.288118,1.340131,0.254273,0.553948,32.0,180.0,85.0,M,...,0.552713,2.237175,0.000722,0.920616,0.363307,0.000888,1.282563,0.614725,0.000267,1.354238
3,-0.470804,-1.801783,8.929937,1.701367,0.193919,0.328752,32.0,180.0,85.0,M,...,0.697146,1.886452,0.000581,0.905243,0.727568,0.000668,1.175494,1.138423,0.000603,1.415926
4,-9.901018,13.441229,7.822155,-0.351828,0.343581,-0.851439,32.0,180.0,85.0,M,...,1.074713,3.329770,0.005098,1.203505,1.706857,0.000565,0.936669,3.093796,0.002285,0.577150
5,-5.150726,6.613287,3.195224,-0.107008,-1.118601,-0.638797,32.0,180.0,85.0,M,...,1.461785,9.083104,0.043227,1.271543,2.602335,0.005232,1.358140,3.267480,0.006812,0.745639
6,-4.931030,5.574285,6.261700,-0.019441,0.012431,0.115133,32.0,180.0,85.0,M,...,1.176317,1.104385,0.000710,1.366146,3.538939,0.009700,1.667364,0.830726,0.003255,1.440714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169385,0.586223,5.443400,1.029336,-0.737727,1.443459,1.335214,23.0,180.0,67.0,M,...,1.343248,7.967362,0.019501,0.917517,3.978832,0.003662,1.133685,9.439387,0.030438,0.913799
169386,2.286937,11.130139,-1.195925,1.260658,-0.457721,-0.981431,23.0,180.0,67.0,M,...,1.502760,7.253110,0.017147,0.899968,6.142548,0.022069,1.258938,8.046604,0.023517,0.994088
169387,-5.076769,14.152212,2.039380,-2.541764,-1.378478,2.412521,23.0,180.0,67.0,M,...,1.377740,5.479225,0.007237,1.008082,3.459003,0.006143,1.726653,5.018624,0.007999,1.087638
169388,1.671382,11.554947,-1.548809,0.596082,-1.174618,-0.679053,23.0,180.0,67.0,M,...,1.449325,7.200013,0.037785,1.189187,3.266761,0.013842,1.523287,7.761068,0.044975,1.151021


In [12]:
y

2         BSC
3         BSC
4         BSC
5         BSC
6         BSC
         ... 
169385    WAL
169386    WAL
169387    WAL
169388    WAL
169389    WAL
Name: label, Length: 167662, dtype: object

In [14]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split with stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify = y)

In [19]:
y_train.value_counts()

label
STD    37368
WAL    36817
SIT     8956
JOG     8653
JUM     8347
LYI     7581
STU     4910
STN     4504
CSO     2335
CSI     2198
SCH     1190
BSC      762
SDL      644
FKL      609
FOL      536
CHU      336
Name: count, dtype: int64

In [20]:
y_test.value_counts()

label
STD    12456
WAL    12273
SIT     2985
JOG     2884
JUM     2783
LYI     2527
STU     1637
STN     1501
CSO      779
CSI      733
SCH      397
BSC      254
SDL      214
FKL      203
FOL      178
CHU      112
Name: count, dtype: int64

In [23]:
df_train.columns

Index(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label', 'age',
       'height', 'weight', 'gender', 'acc_mag', 'gyro_mag', 'acc_x_fft',
       'acc_x_psd', 'acc_x_entropy', 'acc_y_fft', 'acc_y_psd', 'acc_y_entropy',
       'acc_z_fft', 'acc_z_psd', 'acc_z_entropy', 'gyro_x_fft', 'gyro_x_psd',
       'gyro_x_entropy', 'gyro_y_fft', 'gyro_y_psd', 'gyro_y_entropy',
       'gyro_z_fft', 'gyro_z_psd', 'gyro_z_entropy'],
      dtype='object')

In [25]:
# Split features

basic_features = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
subject_features = ['age', 'height', 'weight', 'gender']
time_features = [f for f in df_train.columns if any(suffix in f for suffix in ['psd', 'entropy', 'fft'])]


In [27]:
feature_set_1 = list(set(basic_features))
feature_set_2 = list(set(basic_features + subject_features))
feature_set_3 = list(set(feature_set_2 + time_features))